# Inspect the Point-Source Visibility Simulations

In [22]:
from pyradiosky import SkyModel
from pyuvdata.uvdata import FastUVH5Meta
import numpy as np
from pathlib import Path
import h5py
import sys

In [3]:
sys.path.append("..")

In [4]:
from core import utils

In [5]:
utils.VIS_DIRFMT

'{sky_model}/nt17280-{chunks:03d}chunks'

In [12]:
ptsrc_files = sorted((utils.OUTDIR / utils.VIS_DIRFMT.format(sky_model='ptsrc256', chunks=24)).glob("*_chunk000.uvh5"))

In [14]:
len(ptsrc_files)

349

In [15]:
meta = FastUVH5Meta(ptsrc_files[0])

In [17]:
antpairs = [
    (1, 0),
    (2, 0),
    (4, 0),
    (11, 0),
    (23, 0),
    (52, 0)
]

In [18]:
blidx = [meta.antpairs.index(p) for p in antpairs]

In [20]:
polidx = [meta.pols.index('xx'),meta.pols.index('yy')]

## Frequency-Based Inspection


In [23]:
with h5py.File(ptsrc_files[0],'r') as _fl:
    print(_fl['Data']['visdata'].shape)

(1081440, 1, 4)


In [ ]:
all_data = []

for i, fl in enumerate(ptsrc_files):
    if not f'fch{i:04d}' in fl.stem:
        print(f"Got to {i}")
        break
    
    with h5py.File(fl,'r') as _fl:
        d = _fl['Data']['visdata'][:, 0, polidx]
        d.shape = (meta.Ntimes, meta.Nbls, len(polidx))
        all_data.append(d[:, blidx])
        

In [ ]:
all_data = np.array(all_data)  # shape (Nfreq, Ntime, npol)